In [1]:
!pip install tabula-py

In [2]:
import tabula
import pandas as pd


In [7]:
import tabula
import pandas as pd

# path
file_path_p4 = r"ueh_graduating_score_overview_Dot4_2024.pdf"
file_path_p3 = r"ueh_graduating_score_overview_Dot3_2024.pdf"

# Đọc toàn bộ bảng từ file PDF
dfs = tabula.read_pdf(file_path_p4, pages='all', lattice=True, multiple_tables=True, encoding='utf-8')
dfs_2 = tabula.read_pdf(file_path_p3, pages='all', lattice=True, multiple_tables=True, encoding='utf-8')

# Thêm cột "Đợt xét tốt nghiệp" cho tất cả DataFrames trong mỗi danh sách
for df in dfs:
    df['Đợt xét tốt nghiệp'] = '20241201'

for df in dfs_2:
    df['Đợt xét tốt nghiệp'] = '20240901'

# Gộp tất cả các DataFrames lại thành một DataFrame
combined_df = pd.concat(dfs + dfs_2, ignore_index=True)



In [5]:
dfs

[    Stt Khóa học        Mã SV                Họ và tên  \
 0     1  Khóa 42  31161023018     Cao Nguyễn Quỳnh Chi   
 1     2  Khóa 42  31161022604          Phạm Thị Mỹ Hảo   
 2     3  Khóa 42  31161023629       Bùi Thị Minh Trang   
 3     4  Khóa 42  31161024766           Võ Thị Hải Yến   
 4     5  Khóa 42  31161021795            Lê Thị Nguyên   
 5     6  Khóa 42  31161024722       Dương Thị Thu Thảo   
 6     7  Khóa 42  31161022564         Nguyễn Minh Nhựt   
 7     8  Khóa 42  31161024306          Lê Thị Kim Oanh   
 8     9  Khóa 42  31161023930        Trần Thị Khánh Ly   
 9    10  Khóa 42  31161025424    Nguyễn Thị Minh Nghĩa   
 10   11  Khóa 42  31161022024       Phạm Võ Ánh Nguyệt   
 11   12  Khóa 42  31161024405           Ngô Phương Nhi   
 12   13  Khóa 42  31161025466        Lý Quang Đình Huy   
 13   14  Khóa 42  31161025744             Đinh Thị Huệ   
 14   15  Khóa 42  31161020057          Nguyễn Quang Vũ   
 15   16  Khóa 42  31161023423          Mai Vũ Hoàng My 

In [11]:
# Bước 1: Chuyển đổi xếp loại thành một giá trị số để dễ dàng sắp xếp
# Chuyển đổi thứ tự ưu tiên: Xuất sắc > Giỏi > Khá > Trung bình khá
rank_map = {
    'Xuất sắc': 1,
    'Giỏi': 2,
    'Khá': 3,
    'Trung bình khá': 4
}

combined_df['Xếp loại số'] = combined_df['Xếp loại'].map(rank_map)

# Bước 2: Sắp xếp theo ĐTB TN (giảm dần) và Xếp loại số (tăng dần)
combined_df = combined_df.sort_values(by=['ĐTB TN', 'Xếp loại số'], ascending=[False, True])

# Bước 3: Tạo cột Thứ hạng toàn khóa và Thứ hạng trong chuyên ngành
combined_df['Thứ hạng toàn khóa'] = combined_df.groupby('Khóa học')['ĐTB TN'].rank(method='dense', ascending=False)
combined_df['Thứ hạng trong chuyên ngành'] = combined_df.groupby(['Khóa học', 'Ngành', 'Chuyên ngành'])['ĐTB TN'].rank(method='dense', ascending=False)



In [12]:
combined_df = combined_df.drop(columns=['Xếp loại số'])


In [13]:
combined_df

,Stt,Khóa học,Mã SV,Họ và tên,Ngành,Chuyên ngành,ĐTB TN,Xếp loại,Ghi chú,Đợt xét tốt nghiệp,Thứ hạng toàn khóa,Thứ hạng trong chuyên ngành
3654,316,Khóa 46,31201020731,Dương Tâm Như,Ngôn ngữ Anh,Tiếng Anh thương mại,9.14,Xuất sắc,NaN,20240901,1.0,1.0
248,249,Khóa 46,31201022577,Lê Đức Phát,Tài chính - Ngân hàng,Đầu tư tài chính,9.02,Xuất sắc,NaN,20241201,2.0,1.0
3762,424,Khóa 46 CLC,31201021039,Nguyễn Hồ Xuân Trà,Tài chính - Ngân hàng,Tài chính,8.99,Giỏi,NaN,20240901,1.0,1.0
3672,334,Khóa 46,31191020445,Nguyễn Phước Hảo,Quản trị dịch vụ du lịch và lữ hành,Quản trị lữ hành,8.68,Giỏi,NaN,20240901,3.0,1.0
205,206,Khóa 46,31201020049,Nguyễn Thị Mai Linh,Ngôn ngữ Anh,Tiếng Anh thương mại,8.64,Giỏi,NaN,20241201,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1246,1247,Khóa 47,31211023666,Lê An,Ngôn ngữ Anh,Tiếng Anh thương mại,3.07,Khá,NaN,20241201,90.0,48.0
1238,1239,Khóa 47,31211020131,Bùi Huỳnh Trúc Ly,Ngôn ngữ Anh,Tiếng Anh thương mại,3.02,Khá,NaN,20241201,91.0,49.0
1259,1260,Khóa 47,31211027267,Phạm Thị Thu Thảo,Ngôn ngữ Anh,Tiếng Anh thương mại,2.97,Khá,NaN,20241201,92.0,50.0
1256,1257,Khóa 47,31211026626,Liêu Nguyên Thành,Ngôn ngữ Anh,Tiếng Anh thương mại,2.93,Khá,NaN,20241201,93.0,51.0


In [14]:
combined_df.to_csv("ueh_graduating_score_overview.csv", index=False, encoding='utf-8-sig')